In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from ffa_scraper import FFAScraper
import os
from tqdm import tqdm
import time

In [2]:
def get_data_of_athlete(athlet, athlet_page,skip_years=[2024,2023]):

    date_of_birth = None
    weight = None
    height = None
    sex = None
    events_dates =[]
    events_names =[]
    events_tours = []
    performances = []
    events_locations = []
    years = [*range(2015,2026)]
    for year_to_skip in skip_years:
        years.remove(year_to_skip)
    for i, year in enumerate(years):
        url = athlet_page+f'&saison={year}'
        response = requests.get(url)

# Checking if the request was successful
        if response.status_code == 200:
        # If successful, print the content of the response
            soup = BeautifulSoup(response.text, 'html.parser')
            tables= soup.find_all('table')
            if i == 0:
                date_of_birth = tables[0].find('td', string='Né(e) le').find_next('td').find_next('b').string.strip()
                sex = tables[0].find('td', string='Cat. / Nat.').find_next('td').find_next('b').text.split('/')[1]
            # Extract height and weight (160cm / 56kg)
                height_weight = tables[0].find('td', string='Taille / Poids').find_next('td').find_next('b').string.strip()

            # Split height and weight
                height, weight = height_weight.split(' / ')
            if len(tables) >= 4:
                rows = tables[3].find_all('tr')[1:]
                for row in rows:
                    columns = row.find_all('td')

                    if len(columns) > 1:  # Ensure the row has data columns
                        date = columns[1].text.strip() +f'/{year}'
                        event = columns[2].find('a').text.strip() if columns[2].find('a') else columns[2].text.strip()
                        tour = columns[3].find('a').text.strip() if columns[3].find('a') else columns[3].text.strip()
                        performance = columns[5].text.strip()
                        location = columns[9].find('a').text.strip() if columns[9].find('a') else columns[9].text.strip()

                        events_dates.append(date)
                        events_names.append(event)
                        events_tours.append(tour)
                        performances.append(performance)
                        events_locations.append(location)


        else:
        # If not successful, print the error status
            print(f"Error: {response.status_code}")
    temp_df =pd.DataFrame({'athlte':athlet, 'athlete_page':athlet_page,'sex':sex, 'date_of_birth':date_of_birth, 'weight':weight, 'height':height, 'event_date':events_dates, 'event_name':events_names, 'event_tour':events_tours, 'performence':performances, 'event_location':events_locations})
    return temp_df

In [ ]:
processes_athlet = 0
changes = True
while changes:
    changes = False
    athlet_df = pd.read_csv(r"data/athlets.csv")
    print("Orginal: ", len(athlet_df))

    if os.path.exists('data/raw_master_ffa_results_part1.csv'):
        existing_df = pd.read_csv(r"data/raw_master_ffa_results_part1.csv")
        athlet_df=athlet_df.loc[~athlet_df['athlet_page'].isin(existing_df['athlete_page'].unique())]

    else:
        df = pd.DataFrame({'athlte':[], 'athlete_page':[],'sex':[], 'date_of_birth':[], 'weight':[], 'height':[], 'event_date':[], 'event_name':[], 'event_tour':[], 'performence':[], 'event_location':[]})
        df.to_csv(r'data/raw_master_ffa_results_part1.csv', index=False, mode='w')

    if os.path.exists('data/raw_master_ffa_results_part2.csv'):
        existing_df = pd.read_csv(r"data/raw_master_ffa_results_part2.csv")
        athlet_df=athlet_df.loc[~athlet_df['athlet_page'].isin(existing_df['athlete_page'].unique())]

    else:
        df = pd.DataFrame({'athlte':[], 'athlete_page':[],'sex':[], 'date_of_birth':[], 'weight':[], 'height':[], 'event_date':[], 'event_name':[], 'event_tour':[], 'performence':[], 'event_location':[]})
        df.to_csv(r'data/raw_master_ffa_results_part2.csv', index=False, mode='w')
    if os.path.exists('data/raw_master_ffa_results_part3.csv'):
        existing_df = pd.read_csv(r"data/raw_master_ffa_results_part3.csv")
        athlet_df=athlet_df.loc[~athlet_df['athlet_page'].isin(existing_df['athlete_page'].unique())]

    else:
        df = pd.DataFrame({'athlte':[], 'athlete_page':[],'sex':[], 'date_of_birth':[], 'weight':[], 'height':[], 'event_date':[], 'event_name':[], 'event_tour':[], 'performence':[], 'event_location':[]})
        df.to_csv(r'data/raw_master_ffa_results_part3.csv', index=False, mode='w')

    print("To extract: ",len(athlet_df))
    if len(athlet_df) > 0:
        changes = True
    for index, row in tqdm(athlet_df.iterrows(),total=len(athlet_df)):


        try:

            skip_years =[2024,2023,2022]

            df = get_data_of_athlete(row['athlet'], row['athlet_page'],skip_years=skip_years)
            processes_athlet +=1
        except Exception as e:
            print(e)
            time.sleep(120)
            continue

        df.to_csv(r'data/raw_master_ffa_results_part3.csv', index=False, mode='a', header=False)


Orginal:  64199
To extract:  1469


100%|██████████| 1469/1469 [3:05:16<00:00,  7.57s/it]  


Orginal:  64199
To extract:  0


0it [00:00, ?it/s]


In [4]:
df1 =pd.read_csv(r"data/raw_master_ffa_results_part1.csv")
df2 = pd.read_csv(r"data/raw_master_ffa_results_part2.csv")
df3 = pd.read_csv(r"data\raw_master_ffa_results_part3.csv")

In [5]:
df = pd.concat([df1,df2,df3], ignore_index=True)
df

,athlte,athlete_page,date_of_birth,weight,height,event_date,event_name,event_tour,performence,event_location,sex
0,SALMI Mustapha (Alg),https://bases.athle.fr/asp.net/athletes.aspx?b...,1999,58kg,171cm,13/12/2015,Cadets cam,Cross,13'20'',Evreux,M
1,SALMI Mustapha (Alg),https://bases.athle.fr/asp.net/athletes.aspx?b...,1999,58kg,171cm,11/11/2015,Cadets - challenge a. le targa cam,Cross,13'15'',Louviers,M
2,SALMI Mustapha (Alg),https://bases.athle.fr/asp.net/athletes.aspx?b...,1999,58kg,171cm,04/07/2015,Perche,Final 2,3m82,Chateauroux,M
3,SALMI Mustapha (Alg),https://bases.athle.fr/asp.net/athletes.aspx?b...,1999,58kg,171cm,01/07/2015,Perche,Final 1,3m64,Aulnay Sous Bois,M
4,SALMI Mustapha (Alg),https://bases.athle.fr/asp.net/athletes.aspx?b...,1999,58kg,171cm,27/06/2015,Perche,Final 1,3m76,Caen,M
...,...,...,...,...,...,...,...,...,...,...,...
2551236,BEGOUT Christian,https://bases.athle.fr/asp.net/athletes.aspx?b...,1964,-,-,04/12/2021,Course 20 kms,Trail,2h27'54'',Autrans,M
2551237,BEGOUT Christian,https://bases.athle.fr/asp.net/athletes.aspx?b...,1964,-,-,18/09/2021,Trail de massereau tc,H.Stade,1h16'10'',Sommieres,M
2551238,BEGOUT Christian,https://bases.athle.fr/asp.net/athletes.aspx?b...,1964,-,-,01/01/2021,Les foulées de la biche - 10 km,H.Stade,45:29 (45:15),Saint Gilles,M
2551239,BEGOUT Christian,https://bases.athle.fr/asp.net/athletes.aspx?b...,1964,-,-,13/04/2025,1/2 Marathon,H.Stade,1h41'34'' (1h41'28''),Montpellier,M


In [6]:
df.to_csv("data/raw_master_ffa_results.csv", index=False, header=True)

In [7]:
df.to_csv("data/raw_master_ffa_results_backup.csv", index=False, header=True)